In [1]:
import pandas as pd
import os

# Load raw data from data/ folder
df = pd.read_csv("data/can_matches.csv")
print(f"Loaded {len(df)} matches")
df.head()

Loaded 736 matches


,year,date,team_A,team_B,goals_A,goals_B,stadium
0,1972,23 février 1972,Cameroun,Kenya,2,1,"Stade omnisports, Yaoundé"
1,1972,24 février 1972,Mali,Togo,3,3,"Stade omnisports, Yaoundé"
2,1972,26 février 1972,Mali,Kenya,1,1,"Stade omnisports, Yaoundé"
3,1972,26 février 1972,Cameroun,Togo,2,0,"Stade omnisports, Yaoundé"
4,1972,28 février 1972,Togo,Kenya,1,1,"Stade omnisports, Yaoundé"


## Restructure Columns

In [2]:
df_new = pd.DataFrame()

df_new["HomeTeam"] = df["team_A"]
df_new["AwayTeam"] = df["team_B"]
df_new["Year"] = df["year"]
df_new["HomeGoals"] = df["goals_A"]
df_new["AwayGoals"] = df["goals_B"]

print("Columns restructured")
print(f"shape: {df_new.shape}")
df_new.head()

Columns restructured
shape: (736, 5)


,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,Cameroun,Kenya,1972,2,1
1,Mali,Togo,1972,3,3
2,Mali,Kenya,1972,1,1
3,Cameroun,Togo,1972,2,0
4,Togo,Kenya,1972,1,1


## Fix Goal Scores with Extra Time

In [3]:
# Handle cases like "2 a. p." (after penalty) - extract first number only
for index, row in df_new.iterrows():
    away_goals = row["AwayGoals"]
    if len(away_goals.split(" ")) > 1:
        corrected_away_goals = away_goals.split(" ")[0]
        df_new.at[index, "AwayGoals"] = corrected_away_goals

print("Goal scores cleaned")

Goal scores cleaned


## Manual Corrections for Invalid Entries

In [4]:
# Verify integer values
invalid_entries = []
for index, row in df_new.iterrows():
    try:
        home_goals = int(row["HomeGoals"])
        away_goals = int(row["AwayGoals"])
    except ValueError:
        invalid_entries.append(index)
        print(f"Invalid at index {index}: HomeGoals={row['HomeGoals']}, AwayGoals={row['AwayGoals']}")

if invalid_entries:
    print(f"\nFound {len(invalid_entries)} invalid entries")
else:
    print("All entries are valid!")

Invalid at index 31: HomeGoals=2, AwayGoals=2
a.p.
Invalid at index 65: HomeGoals=2, AwayGoals=0
(tapis

Found 2 invalid entries


In [5]:
# Manually fix problematic entries (from analysis)
if 31 in invalid_entries:
    df_new.at[31, "AwayGoals"] = 2
if 65 in invalid_entries:
    df_new.at[65, "AwayGoals"] = 0

print("✓ Manual corrections applied")

✓ Manual corrections applied


## Add Total Goals Column

In [6]:
df_new["TotalGoals"] = df_new["HomeGoals"].astype(int) + df_new["AwayGoals"].astype(int)

print("Final cleaned data:")
print(df_new.head(10))
print(f"\nShape: {df_new.shape}")

Final cleaned data:
   HomeTeam AwayTeam  Year  HomeGoals AwayGoals  TotalGoals
0  Cameroun    Kenya  1972          2         1           3
1      Mali     Togo  1972          3         3           6
2      Mali    Kenya  1972          1         1           2
3  Cameroun     Togo  1972          2         0           2
4      Togo    Kenya  1972          1         1           2
5  Cameroun     Mali  1972          1         1           2
6     Congo    Maroc  1972          1         1           2
7     Zaïre   Soudan  1972          1         1           2
8     Maroc   Soudan  1972          1         1           2
9     Zaïre    Congo  1972          2         0           2

Shape: (736, 6)


In [8]:
# Etape 1: Analyser les équipes uniques

df_historical_data = df_new.copy()

print("=== ANALYSE DES EQUIPES UNIQUES ===")
print(f"Nombre unique d'equipes Away: {len(df_historical_data['AwayTeam'].unique())}")
print(f"Nombre unique d'equipes Home: {len(df_historical_data['HomeTeam'].unique())}")

unique_away_teams = df_historical_data["AwayTeam"].unique()
unique_home_teams = df_historical_data["HomeTeam"].unique()
all_unique_teams = set(unique_away_teams).union(set(unique_home_teams))

print(f"\nTotal equipes uniques (union): {len(all_unique_teams)}")
print(all_unique_teams)

=== ANALYSE DES EQUIPES UNIQUES ===
Nombre unique d'equipes Away: 55
Nombre unique d'equipes Home: 52

Total equipes uniques (union): 63
{'Mauritanie', 'Zaïre', 'Botswana', 'Tunisie\nVainqueur du premier quart', 'Mozambique', 'Mali\nPerdant de la seconde demi-finale', 'Gabon', 'Tanzanie', "Cote d'ivoire", 'Togo', 'Liberia', 'Gambie', 'Égypte', 'Tunisie\nVainqueur du groupe A', 'Algérie\nSecond du groupe C', 'Maroc\nVainqueur du quatrième quart', 'Burundi', 'Zambie', 'Mali\nVainqueur du deuxième quart', 'RD Congo', 'Burkina Faso', 'Algérie', 'Mali\nVainqueur du groupe B', 'Maroc\nVainqueur du groupe D', 'Kenya', 'Angola', 'Maroc\nVainqueur de la seconde demi-finale', 'Niger', 'Congo', 'Cameroun\nVainqueur du groupe C', 'Ouganda', 'Haute-Volta', 'Libye', 'Namibie', 'Nigeria', 'Nigeria\nVainqueur du troisième quart', 'Bénin', 'Sénégal', "Côte d'Ivoire", 'Éthiopie', 'Malawi', 'Guinée\nSecond du groupe A', 'Soudan', 'Ghana', 'Madagascar', 'Cap-Vert', 'Tunisie', 'Rwanda', 'Maurice', 'Camerou

In [9]:
# Etape 2: Identifier et filtrer les faux noms d'équipes

print("\n=== FILTRAGE DES FAUX NOMS ===")
faux_noms = ["Vainqueur", "\n", "vainqueur", "Second", "seconde", "Perdant"]

print(f"Faux noms a filtrer: {faux_noms}")
print(f"Avant filtrage: {len(all_unique_teams)} equipes")

# Retirer les noms contenant des faux termes
teams_to_remove = []
for team in list(all_unique_teams):
    for faux in faux_noms:
        if faux in team:
            teams_to_remove.append(team)
            break

for team in teams_to_remove:
    all_unique_teams.remove(team)

print(f"Equipes supprimees: {teams_to_remove}")
print(f"Apres filtrage: {len(all_unique_teams)} equipes valides")


=== FILTRAGE DES FAUX NOMS ===
Faux noms a filtrer: ['Vainqueur', '\n', 'vainqueur', 'Second', 'seconde', 'Perdant']
Avant filtrage: 63 equipes
Equipes supprimees: ['Tunisie\nVainqueur du premier quart', 'Mali\nPerdant de la seconde demi-finale', 'Tunisie\nVainqueur du groupe A', 'Algérie\nSecond du groupe C', 'Maroc\nVainqueur du quatrième quart', 'Mali\nVainqueur du deuxième quart', 'Mali\nVainqueur du groupe B', 'Maroc\nVainqueur du groupe D', 'Maroc\nVainqueur de la seconde demi-finale', 'Cameroun\nVainqueur du groupe C', 'Nigeria\nVainqueur du troisième quart', 'Guinée\nSecond du groupe A', 'Nigeria\nPerdant de la première demi-finale', 'Sénégal\nSecond du groupe B', 'Tunisie\nVainqueur de la première demi-finale', 'Nigeria\nSecond du groupe D']
Apres filtrage: 47 equipes valides


In [12]:
print(all_unique_teams)

{'Mauritanie', 'Zaïre', 'Botswana', 'Mozambique', 'Gabon', 'Tanzanie', "Cote d'ivoire", 'Togo', 'Liberia', 'Gambie', 'Égypte', 'Burundi', 'Zambie', 'RD Congo', 'Burkina Faso', 'Algérie', 'Kenya', 'Angola', 'Niger', 'Congo', 'Ouganda', 'Haute-Volta', 'Libye', 'Namibie', 'Nigeria', 'Bénin', 'Sénégal', "Côte d'Ivoire", 'Éthiopie', 'Malawi', 'Soudan', 'Ghana', 'Madagascar', 'Cap-Vert', 'Tunisie', 'Rwanda', 'Maurice', 'Cameroun', 'Guinée-Bissau', 'Zimbabwe', 'Mali', 'Comores', 'Sierra Leone', 'Guinée', 'Afrique du Sud', 'Maroc', 'Guinée équatoriale'}


In [10]:
# Etape 3: Corriger les noms d'équipes invalides dans le DataFrame

print("\n=== CORRECTION DES NOMS D'EQUIPES ===")
corrections_count = 0

for index, row in df_historical_data.iterrows():
    home_team = row["HomeTeam"]
    away_team = row["AwayTeam"]
    
    # Corriger HomeTeam
    if home_team not in all_unique_teams:
        for team in all_unique_teams:
            if team in home_team:
                df_historical_data.at[index, "HomeTeam"] = team
                corrections_count += 1
                break
    
    # Corriger AwayTeam
    if away_team not in all_unique_teams:
        for team in all_unique_teams:
            if team in away_team:
                df_historical_data.at[index, "AwayTeam"] = team
                corrections_count += 1
                break

print(f"Nombre de corrections appliquees: {corrections_count}")


=== CORRECTION DES NOMS D'EQUIPES ===
Nombre de corrections appliquees: 16


In [11]:
# Etape 4: Verifier que toutes les corrections ont ete appliquees

print("\n=== VERIFICATION DES CORRECTIONS ===")
remaining_invalid = 0

for index, row in df_historical_data.iterrows():
    home_team = row["HomeTeam"]
    away_team = row["AwayTeam"]
    
    if home_team not in all_unique_teams:
        print(f"ERREUR - HomeTeam invalide a l'index {index}: {home_team}")
        remaining_invalid += 1
    
    if away_team not in all_unique_teams:
        print(f"ERREUR - AwayTeam invalide a l'index {index}: {away_team}")
        remaining_invalid += 1

if remaining_invalid == 0:
    print("Verification OK: Tous les noms d'equipes sont valides!")
    print(f"Nombre final d'equipes valides: {len(all_unique_teams)}")
else:
    print(f"ATTENTION: {remaining_invalid} entree(s) toujours invalide(s)")


=== VERIFICATION DES CORRECTIONS ===
Verification OK: Tous les noms d'equipes sont valides!
Nombre final d'equipes valides: 47


## Save Final Cleaned Data

In [13]:
# Sauvegarder les donnees nettoyees
print("\n=== SAUVEGARDE DES DONNEES ===")

# Mettre à jour df_new avec les noms d'équipes corrigés
df_new[["HomeTeam", "AwayTeam"]] = df_historical_data[["HomeTeam", "AwayTeam"]]

df_new.to_csv("data/can_matches_cleaned_final.csv", index=False)
print("Fichier sauvegarde: data/can_matches_cleaned_final.csv")
print(f"Total matches nettoyees: {len(df_new)}")
print(f"\nApercu final:")
print(df_new.head(10))


=== SAUVEGARDE DES DONNEES ===
Fichier sauvegarde: data/can_matches_cleaned_final.csv
Total matches nettoyees: 736

Apercu final:
   HomeTeam AwayTeam  Year  HomeGoals AwayGoals  TotalGoals
0  Cameroun    Kenya  1972          2         1           3
1      Mali     Togo  1972          3         3           6
2      Mali    Kenya  1972          1         1           2
3  Cameroun     Togo  1972          2         0           2
4      Togo    Kenya  1972          1         1           2
5  Cameroun     Mali  1972          1         1           2
6     Congo    Maroc  1972          1         1           2
7     Zaïre   Soudan  1972          1         1           2
8     Maroc   Soudan  1972          1         1           2
9     Zaïre    Congo  1972          2         0           2
